In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from autogluon.tabular import TabularPredictor

In [2]:
KNOWN_COLS = [
    # (col_name, legal_feature)
    ("Actual Loops", False),
    ("Actual Input Rows", False),
    ("Actual Rows", False),
    ("Actual Startup Time", False),
    ("Actual Total Time", False),
    ("Alias", True),
    ("Async Capable", True),
    ("Benchmark", False),
    ("Boot", False),
    ("Cache Evictions", False),
    ("Cache Hits", False),
    ("Cache Key", True),
    ("Cache Misses", False),
    ("Cache Mode", True),
    ("Cache Overflows", False),  # TODO(WAN)
    ("Command", True),
    ("CTE Name", True),
    ("DefaultTimeout", False),
    ("Disk Usage", False),
    ("Estimated Input Rows", True),
    ("Exact Heap Blocks", False),
    ("Exclude", False),  # For runtime timeout computation.
    ("Experiment", False),
    ("Filter", True),
    ("Full-sort Groups.Group Count", False),
    ("Full-sort Groups.Sort Methods Used", False),
    ("Full-sort Groups.Sort Space Memory.Average Sort Space Used", False),
    ("Full-sort Groups.Sort Space Memory.Peak Sort Space Used", False),
    ("Group Key", False),  # TODO(WAN)
    ("Grouping Sets", False),  # TODO(WAN)
    ("Hash Batches", False),
    ("Hash Buckets", False),
    ("Hash Cond", True),
    ("HashAgg Batches", False),
    ("Heap Fetches", False),
    ("Index Cond", True),
    ("Index Name", True),
    ("Inner Unique", True),
    ("Join Filter", True),
    ("Join Type", True),
    ("Lossy Heap Blocks", False),
    ("Merge Cond", True),
    ("Node Type", True),
    ("Operator Stopped", False),
    ("Operator Time", False),
    ("Original Hash Batches", False),
    ("Original Hash Buckets", False),
    ("Output", False),  # TODO(WAN)
    ("Parallel Aware", True),
    ("Params Evaluated", False),  # TODO(WAN)
    ("Parent Relationship", True),
    ("Partial Mode", True),
    ("Peak Memory Usage", False),
    ("Plan Node Id", True),
    ("Plan Rows", True),
    ("Plan Width", True),
    ("Planned Partitions", True),
    ("Pre-sorted Groups.Group Count", False),
    ("Pre-sorted Groups.Sort Methods Used", False),
    ("Pre-sorted Groups.Sort Space Memory.Average Sort Space Used", False),
    ("Pre-sorted Groups.Sort Space Memory.Peak Sort Space Used", False),
    ("Presorted Key", False),
    ("Query", False),
    ("Query Execution Time", False),
    ("Query Planning Time", False),
    ("Query Total Time", False),
    ("Recheck Cond", True),
    ("Relation Name", True),
    ("Repeatable Seed", True),
    ("Rows Removed by Filter", False),
    ("Rows Removed by Index Recheck", False),
    ("Rows Removed by Join Filter", False),
    ("Sampling Method", True),
    ("Sampling Parameters", False),  # TODO(WAN)
    ("SF", False),
    ("Scan Direction", True),
    ("Schema", True),
    ("Seed", False),
    ("Single Copy", True),
    ("Sort Key", False),  # TODO(WAN)
    ("Sort Method", True),
    ("Sort Space Type", True),
    ("Sort Space Used", False),
    ("Source", False),
    ("Startup Cost", True),
    ("Strategy", True),
    ("Subplan Name", True),
    ("Subplans Removed", False),
    ("Total Cost", True),
    ("Workers Launched", False),
    ("Workers Planned", True),
]

In [3]:
BENCHMARK = "dsb"
SF = "10"

RAW_DF = pd.read_parquet(f"./data/experiment_{BENCHMARK}_sf_{SF}.pq")
if BENCHMARK == "tpch":
    # https://www.vldb.org/pvldb/vol12/p1733-marcus.pdf
    seeds = RAW_DF["Seed"].unique()
    rng = np.random.default_rng(15721)
    TRAIN_SEEDS = sorted(
        rng.choice(seeds, size=int(0.8 * len(seeds)), replace=False).tolist()
    )
    TEST_SEEDS = sorted(list(set(seeds) - set(TRAIN_SEEDS)))
elif BENCHMARK == "dsb":
    # https://www.vldb.org/pvldb/vol14/p3376-ding.pdf
    TRAIN_SEEDS = ["15721"]
    TEST_SEEDS = ["15722"]
elif BENCHMARK == "job":
    # Follow TPC-H.
    rng = np.random.default_rng(15721)
    seeds = RAW_DF.index.tolist()
    TRAIN_SEEDS = sorted(
        rng.choice(seeds, size=int(0.8 * len(seeds)), replace=False).tolist()
    )
    TEST_SEEDS = sorted(list(set(seeds) - set(TRAIN_SEEDS)))
    RAW_DF.iloc[TRAIN_SEEDS, RAW_DF.columns.get_loc("Seed")] = "seed_train"
    RAW_DF.iloc[TEST_SEEDS, RAW_DF.columns.get_loc("Seed")] = "seed_test"
    TRAIN_SEEDS = ["seed_train"]
    TEST_SEEDS = ["seed_test"]
else:
    raise RuntimeError(f"Unknown: {BENCHMARK}")

TRAIN_DF = RAW_DF[RAW_DF.Seed.isin(TRAIN_SEEDS)].copy()
TEST_DF = RAW_DF[RAW_DF.Seed.isin(TEST_SEEDS)].copy()
print(f"{TRAIN_DF.shape=} {TEST_DF.shape=}")

TRAIN_DF.shape=(5009965, 88) TEST_DF.shape=(121313, 88)


In [4]:
N_TARGETS = 100

ARTIFACT_PATH = Path("./artifact").absolute()
MODEL_BASE_PATH = ARTIFACT_PATH / "model" / BENCHMARK / SF / "base"
MODEL_BASE_PATH.parent.mkdir(parents=True, exist_ok=True)

MODEL_REFINE_PATH = ARTIFACT_PATH / "model" / BENCHMARK / SF / f"refine_{N_TARGETS}"
MODEL_REFINE_PATH.parent.mkdir(parents=True, exist_ok=True)

MODEL_BEST_PATH = ARTIFACT_PATH / "model" / BENCHMARK / SF / "best"
MODEL_BEST_PATH.parent.mkdir(parents=True, exist_ok=True)

In [5]:
SIMULATE_DF = RAW_DF[RAW_DF.Experiment == "default"].groupby(
    ["Experiment", "Benchmark", "SF", "Query", "Seed"]
)


def simulate_execute(benchmark, sf, query, seed):
    return SIMULATE_DF.get_group(("default", benchmark, sf, query, seed))

In [6]:
# Train ML model on some config here.
if not MODEL_BEST_PATH.exists():
    all_features = set(RAW_DF.columns)
    allowed_features = set([feature for feature, allowed in KNOWN_COLS if allowed])
    ignored_columns = sorted(list(all_features - allowed_features))
    allowed_columns = sorted(list(all_features.intersection(allowed_features)))
    print(f"Ignoring {len(ignored_columns)} features: {ignored_columns=}")
    print(f"Allowing {len(allowed_columns)} features: {allowed_columns=}")

    experiment = "default"
    train_df = TRAIN_DF[
        (TRAIN_DF.Experiment == experiment)
        & (TRAIN_DF.Benchmark == BENCHMARK)
        & (TRAIN_DF.SF == SF)
        & (~TRAIN_DF.Exclude)  # Timeouts have no operator time.
    ]

    print(f"Training on: {train_df.shape=}")
    model_best = TabularPredictor(
        label="Operator Time",
        path=str(MODEL_BEST_PATH),
        eval_metric="mean_absolute_error",
        learner_kwargs={"ignored_columns": ignored_columns},
    )
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    model_best.fit(train_df, presets="medium_quality", time_limit=60 * 5)
model_best = TabularPredictor.load(str(MODEL_BEST_PATH))

Ignoring 53 features: ignored_columns=['Actual Input Rows', 'Actual Loops', 'Actual Rows', 'Actual Startup Time', 'Actual Total Time', 'Benchmark', 'Bytejack', 'Cache Evictions', 'Cache Hits', 'Cache Misses', 'Cache Overflows', 'DefaultTimeout', 'Disk Usage', 'Exact Heap Blocks', 'Exclude', 'Experiment', 'Full-sort Groups.Group Count', 'Full-sort Groups.Sort Methods Used', 'Full-sort Groups.Sort Space Memory.Average Sort Space Used', 'Full-sort Groups.Sort Space Memory.Peak Sort Space Used', 'Group Key', 'Grouping Sets', 'Hash Batches', 'Hash Buckets', 'HashAgg Batches', 'Heap Fetches', 'Lossy Heap Blocks', 'Operator Stopped', 'Operator Time', 'Original Hash Batches', 'Original Hash Buckets', 'Output', 'Params Evaluated', 'Peak Memory Usage', 'Pre-sorted Groups.Group Count', 'Pre-sorted Groups.Sort Methods Used', 'Pre-sorted Groups.Sort Space Memory.Average Sort Space Used', 'Pre-sorted Groups.Sort Space Memory.Peak Sort Space Used', 'Presorted Key', 'Query', 'Query Execution Time', 'Q

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #117-Ubuntu SMP Fri Apr 26 12:26:49 UTC 2024
CPU Count:          80
Memory Avail:       173.99 GB / 187.55 GB (92.8%)
Disk Space Avail:   615.13 GB / 879.14 GB (70.0%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "/mnt/nvme1n1/wanshenl/git/surgeon/artifact/model/dsb/10/best"
Train Data Rows:    121013
Train Data Columns: 87
Label Column:       Operator Time
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (153.499, 0.0, 0.06657, 0.85388)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify proble

Training on: train_df.shape=(121013, 88)


Using Feature Generators to preprocess the data ...
Dropping user-specified ignored columns: ['Actual Input Rows', 'Actual Loops', 'Actual Rows', 'Actual Startup Time', 'Actual Total Time', 'Benchmark', 'Bytejack', 'Cache Evictions', 'Cache Hits', 'Cache Misses', 'Cache Overflows', 'DefaultTimeout', 'Disk Usage', 'Exact Heap Blocks', 'Exclude', 'Experiment', 'Full-sort Groups.Group Count', 'Full-sort Groups.Sort Methods Used', 'Full-sort Groups.Sort Space Memory.Average Sort Space Used', 'Full-sort Groups.Sort Space Memory.Peak Sort Space Used', 'Group Key', 'Grouping Sets', 'Hash Batches', 'Hash Buckets', 'HashAgg Batches', 'Heap Fetches', 'Lossy Heap Blocks', 'Operator Stopped', 'Operator Time', 'Original Hash Batches', 'Original Hash Buckets', 'Output', 'Params Evaluated', 'Peak Memory Usage', 'Pre-sorted Groups.Group Count', 'Pre-sorted Groups.Sort Methods Used', 'Pre-sorted Groups.Sort Space Memory.Average Sort Space Used', 'Pre-sorted Groups.Sort Space Memory.Peak Sort Space Used

[1000]	valid_set's l1: 0.0336689
[2000]	valid_set's l1: 0.0327983
[3000]	valid_set's l1: 0.0321245
[4000]	valid_set's l1: 0.031794
[5000]	valid_set's l1: 0.031598
[6000]	valid_set's l1: 0.0314981
[7000]	valid_set's l1: 0.0313362
[8000]	valid_set's l1: 0.0311424
[9000]	valid_set's l1: 0.0310419
[10000]	valid_set's l1: 0.0309781


	-0.031	 = Validation score   (-mean_absolute_error)
	51.24s	 = Training   runtime
	0.22s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 199.56s of the 199.55s of remaining time.


[1000]	valid_set's l1: 0.0324044
[2000]	valid_set's l1: 0.0316745
[3000]	valid_set's l1: 0.0310543
[4000]	valid_set's l1: 0.0306828
[5000]	valid_set's l1: 0.0306928
[6000]	valid_set's l1: 0.0304997
[7000]	valid_set's l1: 0.0303903
[8000]	valid_set's l1: 0.0303802
[9000]	valid_set's l1: 0.0303296
[10000]	valid_set's l1: 0.0302974


	-0.0303	 = Validation score   (-mean_absolute_error)
	48.44s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 150.69s of the 150.68s of remaining time.
	-0.027	 = Validation score   (-mean_absolute_error)
	335.89s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 264.34s of the -186.46s of remaining time.
	Ensemble Weights: {'KNeighborsDist': 1.0}
	-0.0247	 = Validation score   (-mean_absolute_error)
	0.07s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 487.64s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 1728.3 rows/s (2500 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/mnt/nvme1n1/wanshenl/git/surgeon/artifact/model/dsb/10/best")


In [7]:
# Train ML model on some config here.
if not MODEL_BASE_PATH.exists():
    all_features = set(RAW_DF.columns)
    allowed_features = set([feature for feature, allowed in KNOWN_COLS if allowed])
    ignored_columns = sorted(list(all_features - allowed_features))
    allowed_columns = sorted(list(all_features.intersection(allowed_features)))
    print(f"Ignoring {len(ignored_columns)} features: {ignored_columns=}")
    print(f"Allowing {len(allowed_columns)} features: {allowed_columns=}")

    experiment = "bytejack_e1_iea1_ic1_es1_ss1_ssp50_sss15721"
    train_df = TRAIN_DF[
        (TRAIN_DF.Experiment == experiment)
        & (TRAIN_DF.Benchmark == BENCHMARK)
        & (TRAIN_DF.SF == SF)
        & (~TRAIN_DF.Exclude)  # Timeouts have no operator time.
    ]

    print(f"Training on: {train_df.shape=}")
    model_base = TabularPredictor(
        label="Operator Time",
        path=str(MODEL_BASE_PATH),
        eval_metric="mean_absolute_error",
        learner_kwargs={"ignored_columns": ignored_columns},
    )
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    model_base.fit(train_df, presets="medium_quality", time_limit=60 * 5)
model_base = TabularPredictor.load(str(MODEL_BASE_PATH))

Ignoring 53 features: ignored_columns=['Actual Input Rows', 'Actual Loops', 'Actual Rows', 'Actual Startup Time', 'Actual Total Time', 'Benchmark', 'Bytejack', 'Cache Evictions', 'Cache Hits', 'Cache Misses', 'Cache Overflows', 'DefaultTimeout', 'Disk Usage', 'Exact Heap Blocks', 'Exclude', 'Experiment', 'Full-sort Groups.Group Count', 'Full-sort Groups.Sort Methods Used', 'Full-sort Groups.Sort Space Memory.Average Sort Space Used', 'Full-sort Groups.Sort Space Memory.Peak Sort Space Used', 'Group Key', 'Grouping Sets', 'Hash Batches', 'Hash Buckets', 'HashAgg Batches', 'Heap Fetches', 'Lossy Heap Blocks', 'Operator Stopped', 'Operator Time', 'Original Hash Batches', 'Original Hash Buckets', 'Output', 'Params Evaluated', 'Peak Memory Usage', 'Pre-sorted Groups.Group Count', 'Pre-sorted Groups.Sort Methods Used', 'Pre-sorted Groups.Sort Space Memory.Average Sort Space Used', 'Pre-sorted Groups.Sort Space Memory.Peak Sort Space Used', 'Presorted Key', 'Query', 'Query Execution Time', 'Q

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #117-Ubuntu SMP Fri Apr 26 12:26:49 UTC 2024
CPU Count:          80
Memory Avail:       172.44 GB / 187.55 GB (91.9%)
Disk Space Avail:   613.34 GB / 879.14 GB (69.8%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "/mnt/nvme1n1/wanshenl/git/surgeon/artifact/model/dsb/10/base"
Train Data Rows:    125667
Train Data Columns: 87
Label Column:       Operator Time
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (5.08, 1e-06, 0.03756, 0.1643)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify pro

Training on: train_df.shape=(125667, 88)


Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    176568.10 MB
	Train Data (Original)  Memory Usage: 119.94 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 6 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['Join Filter', 'Filter', 'Index Cond', 'Hash Cond']
			CountVectorizer fit with vocabulary size = 551
	Stage 4 Generators:
		Fitting Dro

[1000]	valid_set's l1: 0.0103631
[2000]	valid_set's l1: 0.00943388
[3000]	valid_set's l1: 0.00895853
[4000]	valid_set's l1: 0.00857801
[5000]	valid_set's l1: 0.00829138
[6000]	valid_set's l1: 0.00807025
[7000]	valid_set's l1: 0.00788237
[8000]	valid_set's l1: 0.00773
[9000]	valid_set's l1: 0.0075769
[10000]	valid_set's l1: 0.00744166


	-0.0074	 = Validation score   (-mean_absolute_error)
	51.4s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 217.8s of the 217.8s of remaining time.


[1000]	valid_set's l1: 0.00810925
[2000]	valid_set's l1: 0.00721375
[3000]	valid_set's l1: 0.00681103
[4000]	valid_set's l1: 0.006556
[5000]	valid_set's l1: 0.00635041
[6000]	valid_set's l1: 0.00617671
[7000]	valid_set's l1: 0.00605607
[8000]	valid_set's l1: 0.00592724
[9000]	valid_set's l1: 0.00580345
[10000]	valid_set's l1: 0.0057234


	-0.0057	 = Validation score   (-mean_absolute_error)
	49.41s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 167.77s of the 167.77s of remaining time.
	-0.0046	 = Validation score   (-mean_absolute_error)
	136.6s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 30.77s of the 30.77s of remaining time.
	Ran out of time, early stopping on iteration 616.
	-0.0161	 = Validation score   (-mean_absolute_error)
	31.45s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 275.6s of the -0.77s of remaining time.
	Ensemble Weights: {'KNeighborsDist': 1.0}
	-0.0035	 = Validation score   (-mean_absolute_error)
	0.08s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 301.37s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 3690.1 rows/s (2500 batch size)
Tab

In [8]:
# Eval and fine-tune lenses for ML model here.
test_df = TEST_DF[
    (TEST_DF.Experiment == "default")
    & (TEST_DF.Benchmark == BENCHMARK)
    & (TEST_DF.SF == SF)
    & (~TEST_DF.Exclude)  # Did not train on timeouts, exclude for now.
]
print(f"{test_df.shape=}")

test_df.shape=(121000, 88)


In [9]:
pred_df = model_base.predict(test_df)
pred_df = pd.concat([test_df, pred_df.to_frame(name="Predicted Operator Time")], axis=1)
print(f"{pred_df.shape=}")

pred_df.shape=(121000, 89)


In [10]:
qs_pred_df = pred_df.groupby(["Experiment", "Benchmark", "SF", "Query", "Seed"])
ycol, ypcol = "Operator Time", "Predicted Operator Time"
qs_pred_df = qs_pred_df[[ycol, ypcol]].sum().reset_index()
qs_pred_df["Err"] = qs_pred_df[ycol] - qs_pred_df[ypcol]
qs_pred_df["AbsErr"] = qs_pred_df["Err"].abs()
qs_pred_df["FactorErr"] = (qs_pred_df[ycol] / qs_pred_df[ypcol]).clip(
    lower=(qs_pred_df[ypcol] / qs_pred_df[ycol])
)
qs_pred_df

,Experiment,Benchmark,SF,Query,Seed,Operator Time,Predicted Operator Time,Err,AbsErr,FactorErr
0,default,dsb,10,query001-0-1,15722,0.969,0.575253,0.393747,0.393747,1.684475
1,default,dsb,10,query001-1-1,15722,3.956,0.576453,3.379547,3.379547,6.862654
2,default,dsb,10,query001-11-1,15722,3.739,0.577053,3.161947,3.161947,6.479470
3,default,dsb,10,query001-12-1,15722,57.277,0.594053,56.682947,56.682947,96.417260
4,default,dsb,10,query001-13-1,15722,128.948,0.710328,128.237672,128.237672,181.532921
...,...,...,...,...,...,...,...,...,...,...
4981,default,dsb,10,query102_spj-95-1,15722,3.411,2.436003,0.974997,0.974997,1.400245
4982,default,dsb,10,query102_spj-96-1,15722,0.277,0.481994,-0.204994,0.204994,1.740052
4983,default,dsb,10,query102_spj-97-1,15722,0.176,1.459687,-1.283687,1.283687,8.293678
4984,default,dsb,10,query102_spj-98-1,15722,0.426,0.199048,0.226952,0.226952,2.140186


In [11]:
targets = qs_pred_df.sort_values("AbsErr", ascending=False).head(N_TARGETS)
targets_list = list(targets.groupby(["Benchmark", "SF", "Query", "Seed"]).groups.keys())
targets

,Experiment,Benchmark,SF,Query,Seed,Operator Time,Predicted Operator Time,Err,AbsErr,FactorErr
41,default,dsb,10,query001-50-1,15722,158.586,0.705328,157.880672,157.880672,224.839983
4,default,dsb,10,query001-13-1,15722,128.948,0.710328,128.237672,128.237672,181.532921
19,default,dsb,10,query001-29-1,15722,122.619,0.692729,121.926271,121.926271,177.008599
36,default,dsb,10,query001-46-1,15722,98.646,0.816654,97.829346,97.829346,120.792961
16,default,dsb,10,query001-26-1,15722,88.068,0.617454,87.450546,87.450546,142.630870
4085,default,dsb,10,query092-35-1,15722,66.731,0.338545,66.392455,66.392455,197.111031
9,default,dsb,10,query001-2-1,15722,60.112,0.590653,59.521347,59.521347,101.772034
3,default,dsb,10,query001-12-1,15722,57.277,0.594053,56.682947,56.682947,96.417260
2868,default,dsb,10,query069-84-1,15722,35.534,0.395753,35.138247,35.138247,89.788410
4888,default,dsb,10,query102_spj-10-1,15722,29.083,2.372921,26.710079,26.710079,12.256201


In [12]:
corrections_matcher = pred_df.apply(
    lambda row: (row["Benchmark"], row["SF"], row["Query"], row["Seed"]), axis=1
)
corrections_df = pred_df[corrections_matcher.isin(targets_list)]
corrections_df

,Node Type,Parallel Aware,Async Capable,Startup Cost,Total Cost,Plan Rows,Plan Width,Actual Startup Time,Actual Total Time,Actual Rows,...,Pre-sorted Groups.Sort Space Memory.Average Sort Space Used,Pre-sorted Groups.Sort Space Memory.Peak Sort Space Used,Cache Key,Cache Mode,Params Evaluated,Cache Hits,Cache Misses,Cache Evictions,Cache Overflows,Predicted Operator Time
5008637,Limit,False,False,28449.69,28449.69,1.0,17.0,58614.826,58614.879,0.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.000001
5008638,Sort,False,False,700.61,700.62,1.0,17.0,58614.824,58614.826,0.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.000001
5008639,Nested Loop,False,False,0.85,700.60,1.0,17.0,58614.822,58614.823,0.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.000001
5008640,Nested Loop,False,False,0.42,698.21,1.0,21.0,58614.821,58614.822,0.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.000001
5008641,Seq Scan,False,False,0.00,6.40,8.0,4.0,0.007,0.067,6.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5129201,Sort,False,False,1.27,1.29,10.0,7.0,0.038,0.042,10.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.000001
5129202,Seq Scan,False,False,0.00,1.10,10.0,7.0,0.021,0.025,10.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.000001
5129203,Index Scan,False,False,0.43,3.27,1.0,20.0,0.025,0.025,0.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.014000
5129204,Index Only Scan,False,False,0.14,0.41,3.0,3.0,0.000,0.000,0.0,...,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,0.000001


In [13]:
# Train ML model on some config here.
if not MODEL_REFINE_PATH.exists():
    all_features = set(corrections_df.columns)
    allowed_features = set([feature for feature, allowed in KNOWN_COLS if allowed])
    ignored_columns = sorted(list(all_features - allowed_features))
    allowed_columns = sorted(
        list(all_features.intersection(allowed_features)) + ["Predicted Operator Time"]
    )
    print(f"Ignoring {len(ignored_columns)} features: {ignored_columns=}")
    print(f"Allowing {len(allowed_columns)} features: {allowed_columns=}")

    print(f"Training on: {corrections_df.shape=}")
    model_refine = TabularPredictor(
        label="Operator Time",
        path=str(MODEL_REFINE_PATH),
        eval_metric="mean_absolute_error",
        learner_kwargs={"ignored_columns": ignored_columns},
    )
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    model_refine.fit(corrections_df, presets="medium_quality", time_limit=60 * 5)
model_refine = TabularPredictor.load(str(MODEL_REFINE_PATH))

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #117-Ubuntu SMP Fri Apr 26 12:26:49 UTC 2024
CPU Count:          80
Memory Avail:       171.62 GB / 187.55 GB (91.5%)
Disk Space Avail:   612.57 GB / 879.14 GB (69.7%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "/mnt/nvme1n1/wanshenl/git/surgeon/artifact/model/dsb/10/refine"
Train Data Rows:    1757
Train Data Columns: 88
Label Column:       Operator Time
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (153.25, 0.0, 0.72528, 6.83958)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem

Ignoring 54 features: ignored_columns=['Actual Input Rows', 'Actual Loops', 'Actual Rows', 'Actual Startup Time', 'Actual Total Time', 'Benchmark', 'Bytejack', 'Cache Evictions', 'Cache Hits', 'Cache Misses', 'Cache Overflows', 'DefaultTimeout', 'Disk Usage', 'Exact Heap Blocks', 'Exclude', 'Experiment', 'Full-sort Groups.Group Count', 'Full-sort Groups.Sort Methods Used', 'Full-sort Groups.Sort Space Memory.Average Sort Space Used', 'Full-sort Groups.Sort Space Memory.Peak Sort Space Used', 'Group Key', 'Grouping Sets', 'Hash Batches', 'Hash Buckets', 'HashAgg Batches', 'Heap Fetches', 'Lossy Heap Blocks', 'Operator Stopped', 'Operator Time', 'Original Hash Batches', 'Original Hash Buckets', 'Output', 'Params Evaluated', 'Peak Memory Usage', 'Pre-sorted Groups.Group Count', 'Pre-sorted Groups.Sort Methods Used', 'Pre-sorted Groups.Sort Space Memory.Average Sort Space Used', 'Pre-sorted Groups.Sort Space Memory.Peak Sort Space Used', 'Predicted Operator Time', 'Presorted Key', 'Query',

			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['Filter', 'Index Cond']
			CountVectorizer fit with vocabulary size = 26
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 1): ['Async Capable']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 1): ['Cache Mode']
		These features were not used to generate any of the output features. Add a feature generator compatible with these features to utilize them.
		Features can also be unused if they carry very little information, such 

[1000]	valid_set's l1: 0.892858
[2000]	valid_set's l1: 0.88477
[3000]	valid_set's l1: 0.882881


	-0.8825	 = Validation score   (-mean_absolute_error)
	8.77s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 286.84s of the 286.84s of remaining time.
	-0.7735	 = Validation score   (-mean_absolute_error)
	0.81s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 285.92s of the 285.92s of remaining time.
	-1.0082	 = Validation score   (-mean_absolute_error)
	8.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 277.88s of the 277.88s of remaining time.
	-0.7778	 = Validation score   (-mean_absolute_error)
	0.7s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 277.07s of the 277.07s of remaining time.
	-0.9443	 = Validation score   (-mean_absolute_error)
	4.15s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ... Training model 

[1000]	valid_set's l1: 0.780543


	-0.7789	 = Validation score   (-mean_absolute_error)
	10.73s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 298.16s of the 251.64s of remaining time.
	Ensemble Weights: {'LightGBMLarge': 0.6, 'RandomForestMSE': 0.2, 'XGBoost': 0.2}
	-0.7608	 = Validation score   (-mean_absolute_error)
	0.13s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 48.51s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 3385.9 rows/s (352 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/mnt/nvme1n1/wanshenl/git/surgeon/artifact/model/dsb/10/refine")


In [14]:
pred2_df = model_base.predict(test_df)
pred2_df = pd.concat(
    [test_df, pred2_df.to_frame(name="Predicted Operator Time")], axis=1
)
print(f"{pred2_df.shape=}")

pred2_df.shape=(121000, 89)


In [19]:
pred3_df = model_refine.predict(pred2_df)
pred3_df = pd.concat(
    [pred2_df, pred3_df.to_frame(name="Predicted2 Operator Time")], axis=1
)

predbest_df = model_best.predict(test_df)
predall_df = pd.concat(
    [pred3_df, predbest_df.to_frame(name="PredictedBest Operator Time")], axis=1
)
print(f"{predall_df.shape=}")

predall_df

predall_df.shape=(121000, 91)


,Node Type,Parallel Aware,Async Capable,Startup Cost,Total Cost,Plan Rows,Plan Width,Actual Startup Time,Actual Total Time,Actual Rows,...,Cache Key,Cache Mode,Params Evaluated,Cache Hits,Cache Misses,Cache Evictions,Cache Overflows,Predicted Operator Time,Predicted2 Operator Time,PredictedBest Operator Time
5008580,Limit,False,False,28449.91,28449.92,1.0,17.0,972.569,972.630,0.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,-0.004452,0.0000
5008581,Sort,False,False,700.84,700.84,1.0,17.0,972.568,972.570,0.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,0.003040,0.0000
5008582,Nested Loop,False,False,0.85,700.83,1.0,17.0,972.565,972.567,0.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,-0.000696,0.0000
5008583,Nested Loop,False,False,0.42,698.25,1.0,21.0,972.565,972.566,0.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,-0.000273,0.0000
5008584,Seq Scan,False,False,0.00,6.40,10.0,4.0,0.009,0.058,9.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,0.005556,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5129575,Index Scan,False,False,0.57,0.75,2.0,16.0,0.000,0.000,0.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,0.201749,0.0036
5129576,Index Scan,False,False,0.29,0.31,1.0,8.0,0.000,0.000,0.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,0.107962,0.0000
5129577,Index Scan,False,False,0.14,0.15,1.0,7.0,0.000,0.000,0.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,0.003726,0.0000
5129578,Index Only Scan,False,False,0.14,0.41,3.0,3.0,0.000,0.000,0.0,...,None,None,None,NaN,NaN,NaN,NaN,0.000001,0.004779,0.0000


In [21]:
qs2_pred_df = predall_df.groupby(["Experiment", "Benchmark", "SF", "Query", "Seed"])
ycol, ypcol, yp2col, ypbcol, ypbestcol = (
    "Operator Time",
    "Predicted Operator Time",
    "Predicted2 Operator Time",
    "PredictedB Operator Time",
    "PredictedBest Operator Time",
)
qs2_pred_df = qs2_pred_df[[ycol, ypbestcol, ypcol, yp2col]].sum().reset_index()
qs2_pred_df["ErrBest"] = qs2_pred_df[ycol] - qs2_pred_df[ypbestcol]
qs2_pred_df["AbsErrBest"] = qs2_pred_df["ErrBest"].abs()
qs2_pred_df["FactorErrBest"] = (qs2_pred_df[ycol] / qs2_pred_df[ypbestcol]).clip(
    lower=(qs2_pred_df[ypbestcol] / qs2_pred_df[ycol])
)

qs2_pred_df["Err"] = qs2_pred_df[ycol] - qs2_pred_df[ypcol]
qs2_pred_df["AbsErr"] = qs2_pred_df["Err"].abs()
qs2_pred_df["FactorErr"] = (qs2_pred_df[ycol] / qs2_pred_df[ypcol]).clip(
    lower=(qs2_pred_df[ypcol] / qs2_pred_df[ycol])
)
qs2_pred_df["Err2"] = qs2_pred_df[ycol] - qs2_pred_df[yp2col]
qs2_pred_df["AbsErr2"] = qs2_pred_df["Err2"].abs()
qs2_pred_df["FactorErr2"] = (qs2_pred_df[ycol] / qs2_pred_df[yp2col]).clip(
    lower=(qs2_pred_df[yp2col] / qs2_pred_df[ycol])
)

qs2_pred_df[ypbcol] = qs2_pred_df.apply(
    lambda row: row["Predicted Operator Time"]
    if row["AbsErr"] < row["AbsErr2"]
    else row["Predicted2 Operator Time"],
    axis=1,
)
qs2_pred_df["ErrB"] = qs2_pred_df[ycol] - qs2_pred_df[ypbcol]
qs2_pred_df["AbsErrB"] = qs2_pred_df["ErrB"].abs()
qs2_pred_df["FactorErrB"] = (qs2_pred_df[ycol] / qs2_pred_df[ypbcol]).clip(
    lower=(qs2_pred_df[ypbcol] / qs2_pred_df[ycol])
)

qs2_pred_df.sort_values("AbsErrB", ascending=False)

,Experiment,Benchmark,SF,Query,Seed,Operator Time,PredictedBest Operator Time,Predicted Operator Time,Predicted2 Operator Time,ErrBest,...,Err,AbsErr,FactorErr,Err2,AbsErr2,FactorErr2,PredictedB Operator Time,ErrB,AbsErrB,FactorErrB
41,default,dsb,10,query001-50-1,15722,158.586,2.682600,0.705328,48.403095,155.903400,...,157.880672,157.880672,224.839983,110.182905,110.182905,3.276361,48.403095,110.182905,110.182905,3.276361
4,default,dsb,10,query001-13-1,15722,128.948,2.376267,0.710328,48.401421,126.571733,...,128.237672,128.237672,181.532921,80.546579,80.546579,2.664137,48.401421,80.546579,80.546579,2.664137
19,default,dsb,10,query001-29-1,15722,122.619,2.389600,0.692729,48.393097,120.229400,...,121.926271,121.926271,177.008599,74.225903,74.225903,2.533812,48.393097,74.225903,74.225903,2.533812
36,default,dsb,10,query001-46-1,15722,98.646,2.779600,0.816654,48.392372,95.866400,...,97.829346,97.829346,120.792961,50.253628,50.253628,2.038462,48.392372,50.253628,50.253628,2.038462
16,default,dsb,10,query001-26-1,15722,88.068,2.647200,0.617454,47.873260,85.420800,...,87.450546,87.450546,142.630870,40.194740,40.194740,1.839607,47.873260,40.194740,40.194740,1.839607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2303,default,dsb,10,query054-24-1,15722,0.031,0.036400,0.030823,3.860886,-0.005400,...,0.000177,0.000177,1.005729,-3.829886,3.829886,124.544705,0.030823,0.000177,0.000177,1.005729
1576,default,dsb,10,query031-90-1,15722,0.099,0.100000,0.099137,26.005648,-0.001000,...,-0.000137,0.000137,1.001386,-25.906648,25.906648,262.683310,0.099137,-0.000137,0.000137,1.001386
1328,default,dsb,10,query027_spj-47-1,15722,0.006,0.183600,0.006009,2.722418,-0.177600,...,-0.000009,0.000009,1.001500,-2.716418,2.716418,453.736305,0.006009,-0.000009,0.000009,1.001500
1288,default,dsb,10,query027_spj-10-1,15722,0.006,0.183800,0.006009,2.722418,-0.177800,...,-0.000009,0.000009,1.001500,-2.716418,2.716418,453.736305,0.006009,-0.000009,0.000009,1.001500


In [22]:
print(qs2_pred_df[[ycol, ypcol, yp2col, ypbcol, ypbestcol]].sum())
print(
    qs2_pred_df[[ycol, ypcol, yp2col, ypbcol, ypbestcol]].sum()
    / qs2_pred_df[ycol].sum()
)

Operator Time                    8054.191000
Predicted Operator Time          4543.580566
Predicted2 Operator Time       133365.312500
PredictedB Operator Time         5730.640086
PredictedBest Operator Time      7411.515625
dtype: float64
Operator Time                   1.000000
Predicted Operator Time         0.564126
Predicted2 Operator Time       16.558499
PredictedB Operator Time        0.711510
PredictedBest Operator Time     0.920206
dtype: float64


In [18]:
print(qs2_pred_df["AbsErr"].describe([0.8, 0.9, 0.95]))
print(qs2_pred_df["AbsErr2"].describe([0.8, 0.9, 0.95]))
print(qs2_pred_df["AbsErrB"].describe([0.8, 0.9, 0.95]))

count    3940.000000
mean       38.755737
std        40.478117
min         0.001268
50%        27.677729
80%        57.622522
90%       108.465483
95%       144.062450
max       212.942110
Name: AbsErr, dtype: float64
count    3940.000000
mean      108.382602
std        76.363534
min         0.100224
50%        89.179455
80%       183.782653
90%       248.707752
95%       262.765068
max       279.361476
Name: AbsErr2, dtype: float64
count    3940.000000
mean       25.924588
std        21.478722
min         0.001268
50%        24.238049
80%        46.798371
90%        56.663312
95%        61.447934
max       118.281744
Name: AbsErrB, dtype: float64


In [28]:
with open("./data/raw/default/tpch/sf_100/15721/1-1.res", "r") as f:
    contents = eval("\n".join(f.readlines()))
contents

{'Plan': {'Node Type': 'Aggregate',
  'Strategy': 'Sorted',
  'Partial Mode': 'Finalize',
  'Parallel Aware': False,
  'Async Capable': False,
  'Startup Cost': 14363731.86,
  'Total Cost': 14363740.5,
  'Plan Rows': 6,
  'Plan Width': 236,
  'Actual Startup Time': 69151.745,
  'Actual Total Time': 69168.658,
  'Actual Rows': 4,
  'Actual Loops': 1,
  'Operator Time': 0.0,
  'Operator Stopped': False,
  'Plan Node Id': 0,
  'Output': ['l_returnflag',
   'l_linestatus',
   'sum(l_quantity)',
   'sum(l_extendedprice)',
   "sum((l_extendedprice * ('1'::numeric - l_discount)))",
   "sum(((l_extendedprice * ('1'::numeric - l_discount)) * ('1'::numeric + l_tax)))",
   'avg(l_quantity)',
   'avg(l_extendedprice)',
   'avg(l_discount)',
   'count(*)'],
  'Group Key': ['lineitem.l_returnflag', 'lineitem.l_linestatus'],
  'Plans': [{'Node Type': 'Gather Merge',
    'Parent Relationship': 'Outer',
    'Parallel Aware': False,
    'Async Capable': False,
    'Startup Cost': 14363731.86,
    'Total